# 3. GenieAgent を用いたマルチエージェントシステム

## 概要
- **databricks_langchain.genie.GenieAgent** を使用
- 2つのGenie Spaceを、**エージェントノード**として定義する
- **Supervisor Agent** によるルーティングを行う
- **マルチエージェントアーキテクチャ** の実装

In [0]:
# 必要なパッケージのインストール
%pip install -U -qqq mlflow-skinny[databricks] langgraph==0.3.4 databricks-langchain databricks-agents uv
dbutils.library.restartPython()

## Agent.py の作成（Models From Code）

%%writefile を使用してagent.pyファイルを作成します。これはMLflow Models From Codeで使用されます。

In [0]:
%%writefile agent.py
import functools
import os
from typing import Any, Generator, Literal, Optional

import mlflow
from databricks.sdk import WorkspaceClient
from databricks_langchain import ChatDatabricks
from databricks_langchain.genie import GenieAgent
from langchain_core.runnables import RunnableLambda, RunnableConfig
from langgraph.graph import END, StateGraph
from langgraph.graph.state import CompiledStateGraph
from mlflow.langchain.chat_agent_langgraph import ChatAgentState
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import (
    ChatAgentChunk,
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)
from pydantic import BaseModel

###################################################
## Bakehouse Sales GenieAgent 
###################################################

BAKEHOUSE_GENIE_SPACE_ID = "***"
bakehouse_genie_agent_description = """このスペースは、DatabricksのAI駆動型データ分析ツールであるGenieを使用して、
データベース内の情報を分析するためのものです。ユーザーは、SQLクエリを実行してデータを取得し、分析を行うことができます。
提供されているテーブルには、ベーカリーフランチャイズビジネスのシミュレーションデータが含まれており、
販売トランザクション、顧客情報、フランチャイズ情報、サプライヤー情報、メディアレビューなどが含まれています。"""

def bakehouse_genie_node(state, config: RunnableConfig):
    """Bakehouse Sales Genie Agent node"""
    genie_agent = GenieAgent(
        genie_space_id=BAKEHOUSE_GENIE_SPACE_ID,
        genie_agent_name="Bakehouse_Genie",
        description=bakehouse_genie_agent_description,
        client=WorkspaceClient(
            host=config['configurable'].get("host"),
            token=config['configurable'].get("token"),
        ),
    )

    result = genie_agent.invoke(state)
    return {
        "messages": [
            {
                "role": "assistant",
                "content": result["messages"][-1].content,
                "name": "Bakehouse_Genie",
            }
        ]
    }

###################################################
## Weather Metrics GenieAgent 
###################################################

WEATHER_GENIE_SPACE_ID = "***"
weather_genie_agent_description = """このスペースには、AccuWeatherの気象データを含む2つのテーブルが含まれています。
各テーブルは、トップ50のグローバル都市の1ヶ月分の予測および歴史的気象データを提供します。
データは、温度、湿度、降水量、風速などの気象パラメータを含み、メートル法単位で表されています。"""

def weather_genie_node(state, config: RunnableConfig):
    """Weather Metrics Genie Agent node"""
    genie_agent = GenieAgent(
        genie_space_id=WEATHER_GENIE_SPACE_ID,
        genie_agent_name="Weather_Genie",
        description=weather_genie_agent_description,
        client=WorkspaceClient(
            host=config['configurable'].get("host"),
            token=config['configurable'].get("token"),
        ),
    )

    result = genie_agent.invoke(state)
    return {
        "messages": [
            {
                "role": "assistant",
                "content": result["messages"][-1].content,
                "name": "Weather_Genie",
            }
        ]
    }

#############################
# Supervisor Agent の定義
#############################

# LLMエンドポイント設定
LLM_ENDPOINT_NAME = "databricks-meta-llama-3-3-70b-instruct"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)

# 最大イテレーション数（無限ループ防止）
MAX_ITERATIONS = 3

# Worker descriptions for routing
worker_descriptions = {
    "Bakehouse_Genie": bakehouse_genie_agent_description,
    "Weather_Genie": weather_genie_agent_description,
}

formatted_descriptions = "\n".join(
    f"- {name}: {desc}" for name, desc in worker_descriptions.items()
)

system_prompt = f"""あなたは質問を適切なワーカーエージェントにルーティングするスーパーバイザーです。
以下のワーカーから選択するか、回答が提供された場合は会話を終了してください。

{formatted_descriptions}

質問の内容に基づいて、最も適切なワーカーを選択してください：
- ベーカリー、フランチャイズ、売上、顧客、商品に関する質問 → Bakehouse_Genie
- 天気、気象、温度、湿度、降水量に関する質問 → Weather_Genie
- 回答が得られた場合 → FINISH
"""

options = ["FINISH"] + list(worker_descriptions.keys())
FINISH = {"next_node": "FINISH"}

def supervisor_agent(state):
    """Supervisor agent that routes to appropriate worker"""
    count = state.get("iteration_count", 0) + 1
    if count > MAX_ITERATIONS:
        return FINISH
    
    class NextNode(BaseModel):
        next_node: Literal[tuple(options)]
        reasoning: str = ""

    preprocessor = RunnableLambda(
        lambda state: [{"role": "system", "content": system_prompt}] + state["messages"]
    )
    supervisor_chain = preprocessor | llm.with_structured_output(NextNode)
    
    decision = supervisor_chain.invoke(state)
    next_node = decision.next_node
    
    # 同じノードに2回連続でルーティングされた場合は終了
    if state.get("next_node") == next_node:
        return FINISH
    
    return {
        "iteration_count": count,
        "next_node": next_node
    }

def final_answer(state):
    """最終回答を生成"""
    prompt = """以前のメッセージ内容を使用して、ユーザーの質問に対する最終的な回答を提供してください。
アシスタントメッセージから得られた情報を整理して、わかりやすく回答してください。"""
    
    preprocessor = RunnableLambda(
        lambda state: state["messages"] + [{"role": "user", "content": prompt}]
    )
    final_answer_chain = preprocessor | llm
    
    return {"messages": [final_answer_chain.invoke(state)]}

class AgentState(ChatAgentState):
    """拡張されたエージェント状態"""
    next_node: str
    iteration_count: int

# ワークフローの構築
workflow = StateGraph(AgentState)

# ノードの追加
workflow.add_node("Bakehouse_Genie", bakehouse_genie_node)
workflow.add_node("Weather_Genie", weather_genie_node)
workflow.add_node("supervisor", supervisor_agent)
workflow.add_node("final_answer", final_answer)

# エントリーポイントの設定
workflow.set_entry_point("supervisor")

# ワーカーからsupervisorへのエッジ
for worker in worker_descriptions.keys():
    workflow.add_edge(worker, "supervisor")

# supervisorからの条件付きエッジ
workflow.add_conditional_edges(
    "supervisor",
    lambda x: x["next_node"],
    {**{k: k for k in worker_descriptions.keys()}, "FINISH": "final_answer"},
)

# 最終回答からENDへ
workflow.add_edge("final_answer", END)

# グラフのコンパイル
multi_agent = workflow.compile()

###################################
# ChatAgentラッパーの実装
###################################

class LangGraphChatAgent(ChatAgent):
    """LangGraphをMLflow ChatAgentとしてラップ"""
    
    def __init__(self, agent: CompiledStateGraph):
        self.agent = agent

    def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        """同期的な予測"""
        request = {
            "messages": [m.model_dump_compat(exclude_none=True) for m in messages]
        }

        messages = []
        for event in self.agent.stream(request, stream_mode="updates"):
            for node_data in event.values():
                messages.extend(
                    ChatAgentMessage(**msg) for msg in node_data.get("messages", [])
                )
        return ChatAgentResponse(messages=messages)

    def predict_stream(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> Generator[ChatAgentChunk, None, None]:
        """ストリーミング予測"""
        request = {
            "messages": [m.model_dump_compat(exclude_none=True) for m in messages]
        }
        for event in self.agent.stream(request, stream_mode="updates"):
            for node_data in event.values():
                yield from (
                    ChatAgentChunk(**{"delta": msg})
                    for msg in node_data.get("messages", [])
                )

# エージェントオブジェクトの作成とMLflowへの設定
mlflow.langchain.autolog()
AGENT = LangGraphChatAgent(multi_agent)
mlflow.models.set_model(AGENT)

## エージェントのテスト

In [0]:
dbutils.library.restartPython()

In [0]:
from agent import multi_agent

# エージェントグラフの構造を表示
multi_agent

In [0]:
from agent import AGENT

# Bakehouse関連の質問
response = AGENT.predict(
    {"messages": [{"role": "user", "content": "売上トップ3のフランチャイズ店舗はどこですか？"}]}
)

print("Response:")
for msg in response.messages:
    print(f"\n[{msg.role}]: {msg.content[:500]}..." if len(msg.content) > 500 else f"\n[{msg.role}]: {msg.content}")

In [0]:
# Weather関連の質問
response = AGENT.predict(
    {"messages": [{"role": "user", "content": "東京の2024年7月の平均気温は何度ですか？"}]}
)

print("Response:")
for msg in response.messages:
    print(f"\n[{msg.role}]: {msg.content[:500]}..." if len(msg.content) > 500 else f"\n[{msg.role}]: {msg.content}")

## まとめ

このノートブックでは、以下を実装しました：

1. **databricks_langchain.genie.GenieAgent** を使用した2つのGenie Agentの作成
2. **Supervisor Agent** による自動ルーティング機能
3. **langgraph StateGraph** によるマルチエージェントシステムの構築

### 嬉しさ
- Genie Conversation API は似た関数・クラスが多く実装がやや複雑だが、GenieAgent という高水準 API がそれらをカプセル化してくれるため、開発が楽になる。
- ツール呼び出し（Tool Calling）をサポートしない基盤モデルも利用できる。